In [0]:
silver_df = spark.read.parquet("/Volumes/firstproject1/default/bronze/superstore")

In [0]:
silver_df.printSchema()

In [0]:
silver_df.display()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


**2.Remove records where Ship Date < Order Date**

In [0]:
silver_df = silver_df.filter(col("Ship Date") >= col("Order Date"))

**3.handle missing values in critical columns like Customer ID, Order ID, and Product ID?**

In [0]:
silver_df = silver_df.dropna(subset = ["Order ID","Customer ID","Product ID"])

**Detect and remove duplicate orders based on Order ID + Product ID + Order Date?**

In [0]:
silver_df = silver_df.dropDuplicates(["Order ID","Product ID","Order Date"])

**Trim spaces & standardize casing**

In [0]:
text_cols = [
    "Category", "Sub-Category", "Region",
    "Ship Mode", "Segment", "Country",
    "City", "State"
]

for c in text_cols:
    silver_df = silver_df.withColumn(c, initcap(trim(col(c))))


**Derived Column – Shipping Delay**

In [0]:
silver_df = silver_df.withColumn("Shipping Delay",datediff(col("Ship Date"),col("Order Date")))

**Derived Columns – Order Year & Month**

In [0]:
silver_df = silver_df.withColumn("Order Year",year(col("Order Date")))\
                     .withColumn("Order Month",month(col("Order Date")))

**Negative Profit Validation**

In [0]:
silver_df = silver_df.withColumn("Loss Flag",when(col("Profit") < 0,1).otherwise(0))

In [0]:
silver_df.display()

In [0]:
silver_df.printSchema()

In [0]:
silver_df.write \
    .mode("overwrite") \
    .parquet("/Volumes/firstproject1/default/silver/superstore")
